In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json

import sys

sys.path.append("../")

##################################################################
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"
##################################################################

import logging
from src.utils import logging_utils
from src.utils import env_utils

logger = logging.getLogger(__name__)

logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

import torch
import transformers

logger.info(f"{torch.__version__=}, {torch.version.cuda=}")
logger.info(
    f"{torch.cuda.is_available()=}, {torch.cuda.device_count()=}, {torch.cuda.get_device_name()=}"
)
logger.info(f"{transformers.__version__=}")

/u/asensharma/miniconda3/envs/connection/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-07-31 13:04:32 __main__ INFO     torch.__version__='2.7.1+cu126', torch.version.cuda='12.6'
2025-07-31 13:04:33 __main__ INFO     torch.cuda.is_available()=True, torch.cuda.device_count()=4, torch.cuda.get_device_name()='NVIDIA A100-SXM4-40GB'
2025-07-31 13:04:33 __main__ INFO     transformers.__version__='4.53.2'


In [3]:
from src.utils.training_utils import get_device_map

# model_key = "meta-llama/Llama-3.2-3B"
# model_key = "meta-llama/Llama-3.1-8B"
model_key = "meta-llama/Llama-3.3-70B-Instruct"
# model_key = "meta-llama/Llama-3.1-405B-Instruct"

# model_key = "google/gemma-2-9b-it"
# model_key = "google/gemma-3-12b-it"
# model_key = "google/gemma-2-27b-it"

# model_key = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# model_key = "allenai/OLMo-2-1124-7B-Instruct"
# model_key = "allenai/OLMo-7B-0424-hf"

# model_key = "Qwen/Qwen2-7B"
# model_key = "Qwen/Qwen2.5-14B-Instruct"
# model_key = "Qwen/Qwen2.5-32B-Instruct"
# model_key = "Qwen/Qwen2.5-72B-Instruct"

# model_key = "Qwen/Qwen3-1.7B"
# model_key = "Qwen/Qwen3-4B"
# model_key = "Qwen/Qwen3-8B"
# model_key = "Qwen/Qwen3-14B"
# model_key = "Qwen/Qwen3-32B"

# device_map = get_device_map(model_key, 30, n_gpus=8)
# device_map

2025-07-31 13:04:37 git.cmd DEBUG    Popen(['git', 'version'], cwd=/u/asensharma/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)
2025-07-31 13:04:37 git.cmd DEBUG    Popen(['git', 'version'], cwd=/u/asensharma/Codes/Projects/retrieval/notebooks, stdin=None, shell=False, universal_newlines=False)


In [4]:
# os.environ["BNB_CUDA_VERSION"] = "124"
# ! echo $BNB_CUDA_VERSION
# ! python -m bitsandbytes

In [5]:
from src.models import ModelandTokenizer

# from transformers import BitsAndBytesConfig

mt = ModelandTokenizer(
    model_key=model_key,
    torch_dtype=torch.bfloat16,
    # device_map=device_map,
    device_map="auto",
    # quantization_config = BitsAndBytesConfig(
    #     # load_in_4bit=True
    #     load_in_8bit=True
    # )
    attn_implementation="eager",
)

2025-07-31 13:04:38 src.models WARNING  meta-llama/Llama-3.3-70B-Instruct not found in /projects/bezl/hf_cache
If not found in cache, model will be downloaded from HuggingFace to cache directory
2025-07-31 13:04:38 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443


2025-07-31 13:04:38 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/config.json HTTP/1.1" 200 0
2025-07-31 13:04:38 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
2025-07-31 13:04:38 urllib3.connectionpool DEBUG    https://huggingface.co:443 "GET /api/models/meta-llama/Llama-3.3-70B-Instruct/tree/main/additional_chat_templates?recursive=False&expand=False HTTP/1.1" 404 64


Loading checkpoint shards: 100%|██████████| 30/30 [03:16<00:00,  6.54s/it]

2025-07-31 13:07:56 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/generation_config.json HTTP/1.1" 200 0
2025-07-31 13:07:56 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /meta-llama/Llama-3.3-70B-Instruct/resolve/main/custom_generate/generate.py HTTP/1.1" 404 0


2025-07-31 13:07:57 src.models INFO     loaded model <meta-llama/Llama-3.3-70B-Instruct> | size: 134570.516 MB | dtype: torch.bfloat16 | device: cuda:0


In [8]:
# from src.functional import free_gpu_cache

# # SYNTH_DATASET = "icosahedron_1"
# SYNTH_DATASET = "64"

# checkpoint_path = os.path.join(
#     env_utils.DEFAULT_RESULTS_DIR,
#     "trained_params",
#     f"{SYNTH_DATASET}",
#     "_full__clamp=0.001",
#     model_key.split("/")[-1],
# )

# version = "epoch_1"
# # version = "final_model"

# checkpoint_path = os.path.join(env_utils.DEFAULT_RESULTS_DIR, checkpoint_path, version)

# print(os.listdir(checkpoint_path))

# checkpoint_path = os.path.join(checkpoint_path, "trainable_params.pt")

# loaded_deltas = torch.load(checkpoint_path, map_location="cpu")
# # loaded_deltas

# free_gpu_cache()


# d = loaded_deltas["model<>layers<>5<>mlp<>gate_proj"]
# d.abs().max()

In [9]:
# from src.utils.training_utils import TrainableLM_delta, TrainableLM_LoRA

# #################################################
# Trainable_CLS = TrainableLM_delta
# # Trainable_CLS = TrainableLM_LoRA
# #################################################

# Trainable_CLS.fuse_with_model(mt._model, loaded_deltas)

In [10]:
###############################################################################
LAYERS = [35, 47, 52]
HEADS = [
    (33, 45),
    (33, 18),
    (34, 1),
    (34, 6),
    (34, 7),
    (35, 19),
    (39, 40),
    (42, 30),
    (47, 18),
    (52, 58),
]

# LAYERS = [54, 60, 67]
# HEADS = [
#     (54, 44),
#     (54, 63),
#     (55, 43),
#     (55, 29),
#     (60, 9),
#     (60, 25),
#     (60, 42),
#     (67, 51)
# ]
###############################################################################

In [12]:
from src.selection.data  import load_people_by_category

people_by_category = load_people_by_category(tokenizer = mt.tokenizer)
list(people_by_category.keys())

2025-07-31 13:09:08 src.selection.data INFO     Loaded 16 categories


['actor',
 'singer',
 'comedian',
 'director',
 'basketball player',
 'football player',
 'soccer player',
 'tennis player',
 'golfer',
 'boxer',
 'news anchor',
 'journalist',
 'author',
 'fashion designer',
 'entrepreneur',
 'politician']

In [27]:
from src.selection.data import get_random_sample

sample = get_random_sample(
    people_by_category=people_by_category,
    mt=mt,
    attribute="actor",
    n_distractors=5
)
print(sample.prompt, ">>", sample.obj)

Which person from the following list has their occupation in common with Ryan Reynolds?
Options: David Remnick, Adele, Denzel Washington, Sean Hannity, Jordan Spieth, Karl Lagerfeld.
Ans: >> Denzel Washington


In [52]:
from src.functional import generate_with_patch, predict_next_token

# prompt_template = """Which of these peoples is by profession a {}?
# options: {}
# Answer:"""
# prompt = prompt_template.format(
#     sample.metadata["attribute"],
#     ", ".join(sample.options)
# )

prompt_template = """{}
Who among these people mentioned above is by profession a {}?
Answer:"""

sample.options = ["Norah O'Donnell", "Julia Roberts", "Rory McIlroy", "Petra Kvitová", "Christian Louboutin", "Wes Anderson"]


# prompt = prompt_template.format(
#     ", ".join(sample.options),
#     sample.metadata["attribute"]
# )

last_name_first = []
for name in sample.options:
    first_name = name.split(" ")[0]
    last_name = " ".join(name.split(" ")[1:])
    last_name_first.append(f"{last_name}, {first_name}")

prompt = prompt_template.format(
    "\n".join(f"{idx+1}. {name}" for idx, name in enumerate(last_name_first)),
    sample.metadata["attribute"]
)

# prompt = sample.prompt

print(prompt, " >> ")

gen = generate_with_patch(
    mt=mt,
    inputs=prompt,
    n_gen_per_prompt=1,
    max_new_tokens=20,
    remove_prefix=True,
)[0]
print(gen)

predict_next_token(
    mt = mt,
    inputs=prompt,
)

1. O'Donnell, Norah
2. Roberts, Julia
3. McIlroy, Rory
4. Kvitová, Petra
5. Louboutin, Christian
6. Anderson, Wes
Who among these people mentioned above is by profession a actor?
Answer:  >> 
 2. Roberts, Julia
Julia Roberts is an American actress and producer. She is known


[[PredictedToken(token=' Julia', prob=0.33203125, logit=19.625, token_id=40394, metadata=None),
  PredictedToken(token=' Roberts', prob=0.259765625, logit=19.375, token_id=31248, metadata=None),
  PredictedToken(token=' ', prob=0.138671875, logit=18.75, token_id=220, metadata=None),
  PredictedToken(token=' (', prob=0.051025390625, logit=17.75, token_id=320, metadata=None),
  PredictedToken(token=' Option', prob=0.044921875, logit=17.625, token_id=7104, metadata=None)]]

In [53]:
from src.attention import get_attention_matrices, visualize_average_attn_matrix
from src.functional import interpret_logits
from src.tokens import prepare_input

tokenized = prepare_input(
    tokenizer=mt,
    prompts=prompt,
    add_bos_token="qwen" in mt.name.lower()
)

attn_matrices = get_attention_matrices(
    input=tokenized,
    mt=mt,
    value_weighted=False,
    # patches = []
)

visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=attn_matrices,
    prompt=prompt,
    tokenized=tokenized,
    layer_window=LAYERS,
)

interpret_logits(tokenizer=mt, logits=attn_matrices.logits)

layer=35


--------------------------------------------------------------------------------
layer=47


--------------------------------------------------------------------------------
layer=52


--------------------------------------------------------------------------------


[PredictedToken(token=' Julia', prob=0.33203125, logit=19.625, token_id=40394, metadata=None),
 PredictedToken(token=' Roberts', prob=0.259765625, logit=19.375, token_id=31248, metadata=None),
 PredictedToken(token=' ', prob=0.138671875, logit=18.75, token_id=220, metadata=None),
 PredictedToken(token=' (', prob=0.051025390625, logit=17.75, token_id=320, metadata=None),
 PredictedToken(token=' Option', prob=0.045166015625, logit=17.625, token_id=7104, metadata=None)]

In [54]:
from src.attention import visualize_attn_matrix

# ###############################################################
# N_KV_GROUPS = 1  # individual heads
# # N_KV_GROUPS = mt.config.num_key_value_heads  # grouped heads
# N_HEADS = mt.config.num_attention_heads
# LAYER_RANGE = range(mt.n_layer)
# ###############################################################

# group_id_to_head_indices = {
#     i: [i * N_KV_GROUPS + j for j in range(N_KV_GROUPS)]
#     for i in range(N_HEADS // N_KV_GROUPS)
# }

In [55]:
for layer_idx, head_idx in HEADS:
    print(
        f"Layer: {layer_idx}, Head: {head_idx}"
    )

    head_matrix = torch.Tensor(attn_matrices.attention_matrices[layer_idx, head_idx]).squeeze()
    visualize_attn_matrix(
        attn_matrix=head_matrix,
        tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
        q_index=-1,
        start_from=1
    )

Layer: 33, Head: 45


Layer: 33, Head: 18


Layer: 34, Head: 1


Layer: 34, Head: 6


Layer: 34, Head: 7


Layer: 35, Head: 19


Layer: 39, Head: 40


Layer: 42, Head: 30


Layer: 47, Head: 18


Layer: 52, Head: 58


### Different Attribute

In [56]:
from src.selection.data  import load_people_by_category

people_by_nationality = load_people_by_category(
    tokenizer = mt.tokenizer,
    path = os.path.join(env_utils.DEFAULT_DATA_DIR, "selection_real/nationality.json")
)
list(people_by_nationality.keys())

2025-07-31 13:24:24 src.selection.data INFO     Loaded 16 categories


['Switzerland',
 'UAE',
 'Singapore',
 'Canada',
 'Mexico',
 'Brazil',
 'India',
 'Australia',
 'Israel',
 'South Africa',
 'United Kingdom',
 'Trinidad and Tobago',
 'New Zealand',
 'United States',
 'Belgium',
 'France']

In [57]:
from src.selection.data import get_random_sample

sample = get_random_sample(
    people_by_category=people_by_nationality,
    mt=mt,
    category="nationality",
    attribute="Australia",
    n_distractors=5
)
print(sample.prompt, ">>", sample.obj)

Which person from the following list has their nationality in common with Rose Byrne?
Options: Selena Tan, Leonard Cohen, Etgar Keret, Frida Kahlo, Steve Irwin, Justine Henin.
Ans: >> Steve Irwin


In [58]:
from src.functional import generate_with_patch, predict_next_token

# prompt_template = """Which of these peoples is by profession a {}?
# options: {}
# Answer:"""
# prompt = prompt_template.format(
#     sample.metadata["attribute"],
#     ", ".join(sample.options)
# )

prompt_template = """{}
Who among these people mentioned above is from {}?
Answer:"""
prompt = prompt_template.format(
    ", ".join(sample.options),
    sample.metadata["attribute"]
)

print(prompt, " >> ")

gen = generate_with_patch(
    mt=mt,
    inputs=prompt,
    n_gen_per_prompt=1,
    max_new_tokens=20,
    remove_prefix=True,
)[0]
print(gen)

predict_next_token(
    mt = mt,
    inputs=prompt,
)

Selena Tan, Leonard Cohen, Etgar Keret, Frida Kahlo, Steve Irwin, Justine Henin
Who among these people mentioned above is from Australia?
Answer:  >> 
 Steve Irwin
Steve Irwin, also known as the Crocodile Hunter, was a well-known


[[PredictedToken(token=' Steve', prob=0.796875, logit=20.875, token_id=14129, metadata=None),
  PredictedToken(token=' The', prob=0.083984375, logit=18.625, token_id=578, metadata=None),
  PredictedToken(token=' Only', prob=0.0272216796875, logit=17.5, token_id=8442, metadata=None),
  PredictedToken(token=' Among', prob=0.0113525390625, logit=16.625, token_id=22395, metadata=None),
  PredictedToken(token=' ', prob=0.0113525390625, logit=16.625, token_id=220, metadata=None)]]

In [59]:
from src.attention import get_attention_matrices, visualize_average_attn_matrix
from src.functional import interpret_logits
from src.tokens import prepare_input

tokenized = prepare_input(
    tokenizer=mt,
    prompts=prompt
)

attn_matrices = get_attention_matrices(
    input=tokenized,
    mt=mt,
    value_weighted=False,
    # patches = []
)

visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=attn_matrices,
    prompt=prompt,
    tokenized=tokenized,
    layer_window=LAYERS,
)

interpret_logits(tokenizer=mt, logits=attn_matrices.logits)

layer=35


--------------------------------------------------------------------------------
layer=47


--------------------------------------------------------------------------------
layer=52


--------------------------------------------------------------------------------


[PredictedToken(token=' Steve', prob=0.796875, logit=20.875, token_id=14129, metadata=None),
 PredictedToken(token=' The', prob=0.083984375, logit=18.625, token_id=578, metadata=None),
 PredictedToken(token=' Only', prob=0.0272216796875, logit=17.5, token_id=8442, metadata=None),
 PredictedToken(token=' Among', prob=0.0113525390625, logit=16.625, token_id=22395, metadata=None),
 PredictedToken(token=' ', prob=0.0113525390625, logit=16.625, token_id=220, metadata=None)]

In [60]:
# for group_id in group_id_to_head_indices:
for layer_idx, head_idx in HEADS:
    print(
        f"Layer: {layer_idx}, Head: {head_idx}"
    )

    head_matrix = torch.Tensor(attn_matrices.attention_matrices[layer_idx, head_idx]).squeeze()
    visualize_attn_matrix(
        attn_matrix=head_matrix,
        tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
        q_index=-1,
        start_from=1
    )

Layer: 33, Head: 45


Layer: 33, Head: 18


Layer: 34, Head: 1


Layer: 34, Head: 6


Layer: 34, Head: 7


Layer: 35, Head: 19


Layer: 39, Head: 40


Layer: 42, Head: 30


Layer: 47, Head: 18


Layer: 52, Head: 58


In [71]:
import random

# options = ["Table", "Orange", "Transistor", "Spinach", "Piano", "Coffee"]
# prompt_template = """Which of these objects is by type a {}?
# options: {}
# Answer:"""
# prompt = prompt_template.format(
#     "musical instrument",
#     ", ".join(options)
# )
# print(prompt)

# options = ["anaconda", "python", "cobra", "viper", "mamba", "rattlesnake"]
# prompt = f"""Which of these snake names is also a programming language?
# options: {", ".join(options)}
# Answer:"""

# options = [
#     "The Space Needle",
#     "Louvre Museum",
#     "Colosseum",
#     "Christ the Redeemer",
#     "State of Liberty",
#     "Big Ben",
# ]
# options += ["Eiffel Tower"]
# random.shuffle(options)
country = "England"
prompt_template = """{}
Which of these landmarks is located in {}?
Answer:"""
prompt = prompt_template.format(
    "\n".join([f"{idx+1}. {opt}" for idx, opt in enumerate(options)]), country
)

print(prompt)

gen = generate_with_patch(
    mt=mt,
    inputs=prompt,
    n_gen_per_prompt=1,
    max_new_tokens=20,
    remove_prefix=True,
)[0]
print(gen)
# predict_next_token(
#     mt=mt,
#     inputs=prompt,
# )

1. The Space Needle
2. Louvre Museum
3. Colosseum
4. Christ the Redeemer
5. State of Liberty
6. Big Ben
7. Eiffel Tower
Which of these landmarks is located in England?
Answer:
 Big Ben
Explanation: Big Ben is the nickname for the Great Bell of the clock at the north


In [72]:
from src.attention import get_attention_matrices, visualize_average_attn_matrix
from src.functional import interpret_logits
from src.tokens import prepare_input

tokenized = prepare_input(
    tokenizer=mt,
    prompts=prompt,
    add_bos_token="qwen" in mt.name.lower()
)

attn_matrices = get_attention_matrices(
    input=tokenized,
    mt=mt,
    value_weighted=False,
    # patches = []
)

visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=attn_matrices,
    prompt=prompt,
    tokenized=tokenized,
    layer_window=LAYERS,
)

interpret_logits(tokenizer=mt, logits=attn_matrices.logits)

layer=35


--------------------------------------------------------------------------------
layer=47


--------------------------------------------------------------------------------
layer=52


--------------------------------------------------------------------------------


[PredictedToken(token=' Big', prob=0.68359375, logit=21.75, token_id=6295, metadata=None),
 PredictedToken(token=' ', prob=0.2216796875, logit=20.625, token_id=220, metadata=None),
 PredictedToken(token=' #', prob=0.01422119140625, logit=17.875, token_id=674, metadata=None),
 PredictedToken(token=' Number', prob=0.01251220703125, logit=17.75, token_id=5742, metadata=None),
 PredictedToken(token=' (', prob=0.0067138671875, logit=17.125, token_id=320, metadata=None)]

In [73]:
# for group_id in group_id_to_head_indices:
for layer_idx, head_idx in HEADS:
    print(
        f"Layer: {layer_idx}, Head: {head_idx}"
    )

    head_matrix = torch.Tensor(attn_matrices.attention_matrices[layer_idx, head_idx]).squeeze()
    visualize_attn_matrix(
        attn_matrix=head_matrix,
        tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
        q_index=-1,
        start_from=1
    )

Layer: 33, Head: 45


Layer: 33, Head: 18


Layer: 34, Head: 1


Layer: 34, Head: 6


Layer: 34, Head: 7


Layer: 35, Head: 19


Layer: 39, Head: 40


Layer: 42, Head: 30


Layer: 47, Head: 18


Layer: 52, Head: 58


In [74]:
options = ["Marie Curie", "Albert Einstein", "Grover Cleveland", "Charles Darwin", "Nikola Tesla", "Issac Newton"]
prompt_template = """Which of these people is not a scientist?
options: {}
Answer:"""
prompt = prompt_template.format(
    ", ".join(options)
)
print(prompt)

predict_next_token(
    mt=mt,
    inputs=prompt,
)

tokenized = prepare_input(
    tokenizer=mt,
    prompts=prompt,
    add_bos_token="qwen" in mt.name.lower()
)

attn_matrices = get_attention_matrices(
    input=tokenized,
    mt=mt,
    value_weighted=False,
    # patches = []
)

visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=attn_matrices,
    prompt=prompt,
    tokenized=tokenized,
    layer_window=LAYERS,
)

interpret_logits(tokenizer=mt, logits=attn_matrices.logits)

Which of these people is not a scientist?
options: Marie Curie, Albert Einstein, Grover Cleveland, Charles Darwin, Nikola Tesla, Issac Newton
Answer:
layer=35


--------------------------------------------------------------------------------
layer=47


--------------------------------------------------------------------------------
layer=52


--------------------------------------------------------------------------------


[PredictedToken(token=' Gro', prob=0.9609375, logit=21.25, token_id=18370, metadata=None),
 PredictedToken(token=' The', prob=0.0093994140625, logit=16.625, token_id=578, metadata=None),
 PredictedToken(token=' ', prob=0.0032501220703125, logit=15.5625, token_id=220, metadata=None),
 PredictedToken(token='Gro', prob=0.002532958984375, logit=15.3125, token_id=77435, metadata=None),
 PredictedToken(token=' the', prob=0.0016326904296875, logit=14.875, token_id=279, metadata=None)]

In [75]:
for layer_idx, head_idx in HEADS:
    print(
        f"Layer: {layer_idx}, Head: {head_idx}"
    )

    head_matrix = torch.Tensor(attn_matrices.attention_matrices[layer_idx, head_idx]).squeeze()
    visualize_attn_matrix(
        attn_matrix=head_matrix,
        tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
        q_index=-1,
        start_from=1
    )

Layer: 33, Head: 45


Layer: 33, Head: 18


Layer: 34, Head: 1


Layer: 34, Head: 6


Layer: 34, Head: 7


Layer: 35, Head: 19


Layer: 39, Head: 40


Layer: 42, Head: 30


Layer: 47, Head: 18


Layer: 52, Head: 58


In [76]:
options = ["Michael Jordan", "Serena Williams", "Justin Trudeau", "Mike Tyson", "Carl Sagan", "Tom Cruise"]
prompt_template = """Which of these people is a politician or a scientist?
options: {}
Answer:"""

prompt = prompt_template.format(
    ", ".join(options)
)
print(prompt)
gen = generate_with_patch(
    mt=mt,
    inputs=prompt,
    n_gen_per_prompt=1,
    max_new_tokens=20,
    remove_prefix=True,
)[0]
print(gen)

tokenized = prepare_input(
    tokenizer=mt,
    prompts=prompt,
    add_bos_token="qwen" in mt.name.lower()
)

attn_matrices = get_attention_matrices(
    input=tokenized,
    mt=mt,
    value_weighted=False,
    # patches = []
)

visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=attn_matrices,
    prompt=prompt,
    tokenized=tokenized,
    layer_window=LAYERS,
)

interpret_logits(tokenizer=mt, logits=attn_matrices.logits)

Which of these people is a politician or a scientist?
options: Michael Jordan, Serena Williams, Justin Trudeau, Mike Tyson, Carl Sagan, Tom Cruise
Answer:
 Justin Trudeau and Carl Sagan
Justin Trudeau is a politician and Carl Sagan is a scientist.
layer=35


--------------------------------------------------------------------------------
layer=47


--------------------------------------------------------------------------------
layer=52


--------------------------------------------------------------------------------


[PredictedToken(token=' Justin', prob=0.88671875, logit=20.75, token_id=23278, metadata=None),
 PredictedToken(token=' The', prob=0.023681640625, logit=17.125, token_id=578, metadata=None),
 PredictedToken(token=' Carl', prob=0.0162353515625, logit=16.75, token_id=22770, metadata=None),
 PredictedToken(token=' ', prob=0.00677490234375, logit=15.875, token_id=220, metadata=None),
 PredictedToken(token=' A', prob=0.006378173828125, logit=15.8125, token_id=362, metadata=None)]

In [77]:
for layer_idx, head_idx in HEADS:
    print(
        f"Layer: {layer_idx}, Head: {head_idx}"
    )

    head_matrix = torch.Tensor(attn_matrices.attention_matrices[layer_idx, head_idx]).squeeze()
    visualize_attn_matrix(
        attn_matrix=head_matrix,
        tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
        q_index=-1,
        start_from=1
    )

Layer: 33, Head: 45


Layer: 33, Head: 18


Layer: 34, Head: 1


Layer: 34, Head: 6


Layer: 34, Head: 7


Layer: 35, Head: 19


Layer: 39, Head: 40


Layer: 42, Head: 30


Layer: 47, Head: 18


Layer: 52, Head: 58


In [82]:
options = ["46", "57", "55", "62", "39"]
# prompt_template = """Which of these numbers is a multiple of 3?
# options: {}
# Answer:"""

prompt_template = """{}
Which of these numbers is a multiple of 3?
Ans:"""

prompt = prompt_template.format(
    ", ".join(options)
)
print(prompt)

gen = generate_with_patch(
    mt=mt,
    inputs=prompt,
    n_gen_per_prompt=1,
    max_new_tokens=20,
    remove_prefix=True,
)[0]
print(gen)
tokenized = prepare_input(
    tokenizer=mt,
    prompts=prompt,
    add_bos_token="qwen" in mt.name.lower()
)
attn_matrices = get_attention_matrices(
    input=tokenized,
    mt=mt,
    value_weighted=False,
    # patches = []
)
visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=attn_matrices,
    prompt=prompt,
    tokenized=tokenized,
    layer_window=LAYERS,
)
interpret_logits(tokenizer=mt, logits=attn_matrices.logits)

46, 57, 55, 62, 39
Which of these numbers is a multiple of 3?
Ans:
 57
In the above numbers 57 is the only number which is a multiple of 3
layer=35


--------------------------------------------------------------------------------
layer=47


--------------------------------------------------------------------------------
layer=52


--------------------------------------------------------------------------------


[PredictedToken(token=' ', prob=0.80859375, logit=19.5, token_id=220, metadata=None),
 PredictedToken(token=' The', prob=0.03125, logit=16.25, token_id=578, metadata=None),
 PredictedToken(token=' A', prob=0.03125, logit=16.25, token_id=362, metadata=None),
 PredictedToken(token=' To', prob=0.0147705078125, logit=15.5, token_id=2057, metadata=None),
 PredictedToken(token=' All', prob=0.006988525390625, logit=14.75, token_id=2052, metadata=None)]

In [83]:
# for group_id in group_id_to_head_indices:
for layer_idx, head_idx in HEADS:
    print(
        f"Layer: {layer_idx}, Head: {head_idx}"
    )

    head_matrix = torch.Tensor(attn_matrices.attention_matrices[layer_idx, head_idx]).squeeze()
    visualize_attn_matrix(
        attn_matrix=head_matrix,
        tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
        q_index=-1,
        start_from=1
    )

Layer: 33, Head: 45


Layer: 33, Head: 18


Layer: 34, Head: 1


Layer: 34, Head: 6


Layer: 34, Head: 7


Layer: 35, Head: 19


Layer: 39, Head: 40


Layer: 42, Head: 30


Layer: 47, Head: 18


Layer: 52, Head: 58


In [89]:
# NOVELTY: Find the odd one out

# options = ["Issac Newton", "Albert Einstein", "Marie Curie", "Hugh Jackman", "Charles Darwin"]
options = [
    "Brad Pitt",
    "Tom Hanks",
    "Leonardo DiCaprio",
    "Tom Brady",
    "Daniel Radcliffe",
    "Hugh Jackman",
]

# prompt_template = """Which person from the following list is different from the others?
# {}.
# Ans:"""
prompt_template = """{}
Which among these people are different from others?
Answer:"""
# prompt = prompt_template.format(", ".join(options))
prompt = prompt_template.format("\n".join([f"{idx+1}. {name}" for idx, name in enumerate(options)]))

print(prompt)
gen = generate_with_patch(
    mt=mt,
    inputs=prompt,
    n_gen_per_prompt=1,
    max_new_tokens=20,
    remove_prefix=True,
)[0]
print(gen)
tokenized = prepare_input(
    tokenizer=mt, prompts=prompt, add_bos_token="qwen" in mt.name.lower()
)
attn_matrices = get_attention_matrices(
    input=tokenized,
    mt=mt,
    value_weighted=False,
    # patches = []
)
visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=attn_matrices,
    prompt=prompt,
    tokenized=tokenized,
    layer_window=LAYERS,
)
interpret_logits(tokenizer=mt, logits=attn_matrices.logits)

1. Brad Pitt
2. Tom Hanks
3. Leonardo DiCaprio
4. Tom Brady
5. Daniel Radcliffe
6. Hugh Jackman
Which among these people are different from others?
Answer:
 Tom Brady
The others are all actors while Tom Brady is a football player.
layer=35


--------------------------------------------------------------------------------
layer=47


--------------------------------------------------------------------------------
layer=52


--------------------------------------------------------------------------------


[PredictedToken(token=' Daniel', prob=0.3671875, logit=18.625, token_id=15469, metadata=None),
 PredictedToken(token=' Tom', prob=0.32421875, logit=18.5, token_id=8529, metadata=None),
 PredictedToken(token=' ', prob=0.06396484375, logit=16.875, token_id=220, metadata=None),
 PredictedToken(token=' Hugh', prob=0.056396484375, logit=16.75, token_id=30206, metadata=None),
 PredictedToken(token=' The', prob=0.0250244140625, logit=15.9375, token_id=578, metadata=None)]

In [90]:
for layer_idx, head_idx in HEADS:
    print(
        f"Layer: {layer_idx}, Head: {head_idx}"
    )

    head_matrix = torch.Tensor(attn_matrices.attention_matrices[layer_idx, head_idx]).squeeze()
    visualize_attn_matrix(
        attn_matrix=head_matrix,
        tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
        q_index=-1,
        start_from=1
    )

Layer: 33, Head: 45


Layer: 33, Head: 18


Layer: 34, Head: 1


Layer: 34, Head: 6


Layer: 34, Head: 7


Layer: 35, Head: 19


Layer: 39, Head: 40


Layer: 42, Head: 30


Layer: 47, Head: 18


Layer: 52, Head: 58


In [113]:
# rhymes with

options = ["orange", "mat", "book", "rabbit", "bowl", "watch", "mirror"]
word = "rat"
# word = "look"
# word = "fetch"
# word = "growl"

prompt_template = """{}
Which of these words rhymes with the word "{}"?
Answer:"""
# prompt = prompt_template.format(
#     ", ".join(options),
#     word
# )
prompt = prompt_template.format("\n".join([f"{idx+1}. {name}" for idx, name in enumerate(options)]), word)


# prompt_template = """Which of the following words rhymes with {}
# {}
# Answer:"""
# prompt = prompt_template.format(word, "\n".join([f"{idx+1}. {name}" for idx, name in enumerate(options)]))


print(prompt)
gen = generate_with_patch(
    mt=mt,
    inputs=prompt,
    n_gen_per_prompt=1,
    max_new_tokens=20,
    remove_prefix=True,
)[0]
print(gen)
tokenized = prepare_input(tokenizer=mt, prompts=prompt, add_bos_token="qwen" in mt.name.lower())
attn_matrices = get_attention_matrices(
    input=tokenized,
    mt=mt,
    value_weighted=False,
    # patches = []
)
visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=attn_matrices,
    prompt=prompt,
    tokenized=tokenized,
    layer_window=LAYERS,
)
interpret_logits(tokenizer=mt, logits=attn_matrices.logits)

1. orange
2. mat
3. book
4. rabbit
5. bowl
6. watch
7. mirror
Which of these words rhymes with the word "rat"?
Answer:
 mat
Which of these words refers to a type of fruit?
Answer: orange
Which of these
layer=35


--------------------------------------------------------------------------------
layer=47


--------------------------------------------------------------------------------
layer=52


--------------------------------------------------------------------------------


[PredictedToken(token=' mat', prob=0.6796875, logit=20.75, token_id=5634, metadata=None),
 PredictedToken(token=' The', prob=0.055908203125, logit=18.25, token_id=578, metadata=None),
 PredictedToken(token=' ', prob=0.04345703125, logit=18.0, token_id=220, metadata=None),
 PredictedToken(token=' **', prob=0.033935546875, logit=17.75, token_id=3146, metadata=None),
 PredictedToken(token=' None', prob=0.0299072265625, logit=17.625, token_id=2290, metadata=None)]

In [114]:
for layer_idx, head_idx in HEADS:
    print(
        f"Layer: {layer_idx}, Head: {head_idx}"
    )

    head_matrix = torch.Tensor(attn_matrices.attention_matrices[layer_idx, head_idx]).squeeze()
    visualize_attn_matrix(
        attn_matrix=head_matrix,
        tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
        q_index=-1,
        start_from=1
    )

Layer: 33, Head: 45


Layer: 33, Head: 18


Layer: 34, Head: 1


Layer: 34, Head: 6


Layer: 34, Head: 7


Layer: 35, Head: 19


Layer: 39, Head: 40


Layer: 42, Head: 30


Layer: 47, Head: 18


Layer: 52, Head: 58


In [94]:
#! Logical deduction

# prompt = """John is taller than Mary. Mary is taller than Steve. Who is the shortest?
# Ans:"""
prompt = """John is taller than Bob, but shorter than Steve. Who is the shortest?
Ans:"""
# prompt = """All the cookies are either chocolate or vanilla. None of the chocolate cookies have nuts.
# This cookie has nuts. What flavor is it?
# Ans:"""
# prompt = """The red box is to the left of the blue box, and to the right of the green box.
# Which is the rightmost box?
# Ans:"""

gen = generate_with_patch(
    mt=mt,
    inputs=prompt,
    n_gen_per_prompt=1,
    max_new_tokens=50,
    remove_prefix=True,
)[0]
print(f'"{gen}"')

tokenized = prepare_input(
    tokenizer=mt, prompts=prompt, add_bos_token="qwen" in mt.name.lower()
)
attn_matrices = get_attention_matrices(
    input=tokenized,
    mt=mt,
    value_weighted=False,
    # patches = []
)
visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=attn_matrices,
    prompt=prompt,
    tokenized=tokenized,
    layer_window=LAYERS,
)
interpret_logits(tokenizer=mt, logits=attn_matrices.logits)

" Bob, because John is taller than Bob.
John is taller than Bob, but shorter than Steve. Who is the tallest?
Ans: Steve, because John is shorter than Steve.
John is taller than Bob, but shorter than Steve. Who is in"
layer=35


--------------------------------------------------------------------------------
layer=47


--------------------------------------------------------------------------------
layer=52


--------------------------------------------------------------------------------


[PredictedToken(token=' Bob', prob=0.8203125, logit=20.25, token_id=14596, metadata=None),
 PredictedToken(token=' John', prob=0.03173828125, logit=17.0, token_id=3842, metadata=None),
 PredictedToken(token=' We', prob=0.0218505859375, logit=16.625, token_id=1226, metadata=None),
 PredictedToken(token=' The', prob=0.019287109375, logit=16.5, token_id=578, metadata=None),
 PredictedToken(token=' Since', prob=0.0150146484375, logit=16.25, token_id=8876, metadata=None)]

In [95]:
for layer_idx, head_idx in HEADS:
    print(
        f"Layer: {layer_idx}, Head: {head_idx}"
    )

    head_matrix = torch.Tensor(attn_matrices.attention_matrices[layer_idx, head_idx]).squeeze()
    visualize_attn_matrix(
        attn_matrix=head_matrix,
        tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
        q_index=-1,
        start_from=1
    )

Layer: 33, Head: 45


Layer: 33, Head: 18


Layer: 34, Head: 1


Layer: 34, Head: 6


Layer: 34, Head: 7


Layer: 35, Head: 19


Layer: 39, Head: 40


Layer: 42, Head: 30


Layer: 47, Head: 18


Layer: 52, Head: 58


In [108]:
prompt = """x = True
y = False
z = True

a = x and y
b = y or z
c = z xor x

Which one among a, b, c is True?
Ans:"""

# prompt = """a, b, c is calculated as below. Find which one among a, b, c is True

# x = True
# y = False
# z = True

# a = x and y
# b = y or z
# c = z xor x

# Ans:"""

gen = generate_with_patch(
    mt=mt,
    inputs=prompt,
    n_gen_per_prompt=1,
    max_new_tokens=50,
    remove_prefix=True,
)[0]
print(f'"{gen}"')

tokenized = prepare_input(tokenizer=mt, prompts=prompt, add_bos_token="qwen" in mt.name.lower())
attn_matrices = get_attention_matrices(
    input=tokenized,
    mt=mt,
    value_weighted=False,
    # patches = []
)
visualize_average_attn_matrix(
    mt=mt,
    attn_matrices=attn_matrices,
    prompt=prompt,
    tokenized=tokenized,
    layer_window=LAYERS,
)
interpret_logits(tokenizer=mt, logits=attn_matrices.logits)

" b

Explanation:
a = x and y
   = True and False
   = False

b = y or z
   = False or True
   = True

c = z xor x
   = True xor True
   ="
layer=35


--------------------------------------------------------------------------------
layer=47


--------------------------------------------------------------------------------
layer=52


--------------------------------------------------------------------------------


[PredictedToken(token=' b', prob=0.7421875, logit=20.375, token_id=293, metadata=None),
 PredictedToken(token=' Only', prob=0.07861328125, logit=18.125, token_id=8442, metadata=None),
 PredictedToken(token=' \n', prob=0.0419921875, logit=17.5, token_id=720, metadata=None),
 PredictedToken(token=' B', prob=0.0224609375, logit=16.875, token_id=426, metadata=None),
 PredictedToken(token=' a', prob=0.01544189453125, logit=16.5, token_id=264, metadata=None)]

In [107]:
for layer_idx, head_idx in HEADS:
    print(
        f"Layer: {layer_idx}, Head: {head_idx}"
    )

    head_matrix = torch.Tensor(attn_matrices.attention_matrices[layer_idx, head_idx]).squeeze()
    visualize_attn_matrix(
        attn_matrix=head_matrix,
        tokens=[mt.tokenizer.decode(t) for t in tokenized.input_ids[0]],
        q_index=-1,
        start_from=1
    )

Layer: 33, Head: 45


Layer: 33, Head: 18


Layer: 34, Head: 1


Layer: 34, Head: 6


Layer: 34, Head: 7


Layer: 35, Head: 19


Layer: 39, Head: 40


Layer: 42, Head: 30


Layer: 47, Head: 18


Layer: 52, Head: 58
